In [1]:
#import libraries
import numpy as np 
import pandas as pd
from pgmpy.models import BayesianNetwork


In [2]:
#import data
kunden_train = pd.read_table("data_dmc2002_train.txt")
kunden_test = pd.read_table("data_dmc2002_class.txt")
kunden_realclass = pd.read_table("data_dmc2002_realclass.txt")

In [3]:
#data cleaning & preprocessing of training data
kunden_train.drop_duplicates
kunden_train=kunden_train.drop(columns= ["payment_type","power_consumption","HHH","HGEW",
                                         "MTREG0G","MTKAU0G","MTSTR0G","MTADE0G","MTKDI0G","MTKLE0G","MTKKL0G","MTKGB0G","MTKGL0G",
                                         "PHARM1","PHARM2","PHARM3","PHARM4","PHARM5","PHARM6"], axis= 1)
#filling missing values with median of training data 
kunden_train = kunden_train.fillna(kunden_train.median(numeric_only=True))

#data cleaning & preprocessing of test data
kunden_test.drop_duplicates
kunden_test=kunden_test.drop(columns= ["payment_type","power_consumption","HHH","HGEW","MTREG0G",
                                        "MTKAU0G","MTSTR0G","MTADE0G","MTKDI0G","MTKLE0G","MTKKL0G","MTKGB0G","MTKGL0G",
                                        "PHARM1","PHARM2","PHARM3","PHARM4","PHARM5","PHARM6"], axis= 1)
#filling missing values with median of test data 
kunden_test = kunden_test.fillna(kunden_train.median(numeric_only=True))


In [4]:
model = BayesianNetwork([('SCMWGR2', 'MTBON0G'),('SCMWGR3', 'MTBON0G'),('SCMWGR4', 'MTBON0G'),('SCMWGR5', 'MTBON0G'),('SCMWGR6', 'MTBON0G'),('SCMWGR7', 'MTBON0G'),('SCMWGR21', 'MTBON0G'),('SCMWGR22', 'MTBON0G'),
                         ('MTSTA0G', 'MTBON0G'),('MTSTA0G','MTBEB0G'),('MTSTA0G','canceler'),('MTBEB0G','canceler'),('MTALT0G','canceler'),('MTALT0G','MTFAM0G')])

In [5]:
from pgmpy.estimators import MaximumLikelihoodEstimator
mle = MaximumLikelihoodEstimator(model, kunden_train)

+--------------+--------+
| SCMWGR2(1.0) | 0.1774 |
+--------------+--------+
| SCMWGR2(2.0) | 0.1317 |
+--------------+--------+
| SCMWGR2(3.0) | 0.3726 |
+--------------+--------+
| SCMWGR2(4.0) | 0.12   |
+--------------+--------+
| SCMWGR2(5.0) | 0.1983 |
+--------------+--------+
+--------------+--------------------+-----+--------------------+--------------------+
| MTSTA0G      | MTSTA0G(1.0)       | ... | MTSTA0G(9.0)       | MTSTA0G(9.0)       |
+--------------+--------------------+-----+--------------------+--------------------+
| SCMWGR2      | SCMWGR2(1.0)       | ... | SCMWGR2(5.0)       | SCMWGR2(5.0)       |
+--------------+--------------------+-----+--------------------+--------------------+
| SCMWGR21     | SCMWGR21(1.0)      | ... | SCMWGR21(5.0)      | SCMWGR21(5.0)      |
+--------------+--------------------+-----+--------------------+--------------------+
| SCMWGR22     | SCMWGR22(1.0)      | ... | SCMWGR22(5.0)      | SCMWGR22(5.0)      |
+--------------+----------

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

[]